In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import warnings
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from collections import Counter
from collections import defaultdict
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
import math
from konlpy.tag import Twitter
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer,LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tag import untag
from nltk import Text
from matplotlib import pyplot as plt
from nltk import FreqDist

warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/kimEn/Desktop", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver = webdriver.Chrome('chromedriver.exe', options=options)

driver.get('https://paperswithcode.com/sota')

In [2]:
#맨처음 목록 목차 분류작업

class_2b_url=[]  #big 분류페이지로
class0=[] #class_big와 길이확인**

#browse sota대분류 class0 -  중분류 class1  소분류 -pdf설명페이지-pdf다운 길이확인ㅇ
#class0=맨처음페이지 대분류 리스트, class0_url=대분류페이지 이동url
list1=driver.find_elements(By.CLASS_NAME,'col-md-12 > h4 > a')
for i in list1:
    class0.append(i.text)
    class_2b_url.append(i.get_attribute('href'))

In [3]:
#처음목차와 다음 목차 사전 저장& 넘어갈 url수집
dic_bAndm={}
dic_mAnds={}
class_big=[]
class1=[]  #class m과 길이비교**
class_b2m_url=[]
class_small=[]
class_m2s_url=[]
class_small_dic=[]
tmp=[]
mid_url=[]

#대분류url로 이동
for i in range(len(class_2b_url)):
    driver.get(class_2b_url[i]) 
    
#class1=computer vision하위 518개 
#class1=대분류 내 중분류 리스트, class1_url=see all 클릭시 넘어가는 url / none일경우 소분류로 넘어가기 추가**** 
    list1=driver.find_elements(By.CLASS_NAME,'infinite-container.featured-task > div.row > div.col-md-12 > h2')
    list4=driver.find_elements(By.CLASS_NAME, 'col-lg-12 > h1') #dic1&2(대-중분류) key
    list3=driver.find_elements(By.CLASS_NAME,'col-xl-8.card-col.card-col-title > h1') #소분류text
    
    for i in list4:
        class_big.append(i.text)    
        for j in list1:             
            tmp.append(j.text)
            dic_bAndm[i.text]=tmp  #대-중dictionary(list형식)
        tmp=[]
        
#         for k in list3:
#             class_small_dic.append(k.text)
#             dic_mAnds[class1]=class_small_dic   #중-소분류 사전 수정
    
    #mid_url = 중분류에서 바로 소분류로 넘어가는 url
    mid_url_elem=driver.find_elements(By.CLASS_NAME,'card > a')
    for i in mid_url_elem:
        if i not in mid_url:
            mid_url.append(i.get_attribute('href'))
    
    list2 = driver.find_elements(By.CLASS_NAME,'sota-all-tasks > a')
    for i in list2:          
            class_b2m_url.append(i.get_attribute('href'))



In [4]:
#중분류 모두보기 있는 경우 앞과 겹치지 않게 url 수집 mid_url에 저장

for i in range(len(class_b2m_url)):
    driver.get(class_b2m_url[i])
    list4=driver.find_elements(By.CLASS_NAME,'card-title')
    list5=driver.find_elements(By.CLASS_NAME,'card > a')
    
#      #소분류
#     for i in list4:
#         if i not in list4:
#             class_small.append(i.text)
     
    #중분류 모두보기에서 소분류 url 수집
    for i in list5:
        if i not in mid_url:
            mid_url.append(i.get_attribute('href'))



In [5]:
#목록 df저장/csv저장-없는 것만 진행

In [6]:
# 논문 제목, 상세 url 수집
# classname>css section
    
title = []
url = []
pdf=[]

#mid_url(소분류 url) 반복하면서 title, url 리스트 생성
for i in mid_url:
    driver.get(f'{i}')

#title, url
    Tlist=driver.find_elements(By.CLASS_NAME,'col-lg-9.item-content > h1 > a')
    for i in Tlist:
        title.append(i.text)
        url.append(i.get_attribute('href'))

In [12]:
#pdf_url 수집
for i in range(len(url)):
    
    driver.get(url[i])
    pdf_url=driver.find_elements(By.CLASS_NAME,'badge.badge-light')[0].get_attribute('href')
    
    if pdf_url not in pdf:
        pdf.append(pdf_url)

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x00639943+2595139]
	Ordinal0 [0x005CC9F1+2148849]
	Ordinal0 [0x004C4528+1066280]
	Ordinal0 [0x004B69BE+1010110]
	Ordinal0 [0x004B65AF+1009071]
	Ordinal0 [0x004B5BC6+1006534]
	Ordinal0 [0x004B4AD0+1002192]
	Ordinal0 [0x004B50D8+1003736]
	Ordinal0 [0x004C0AAC+1051308]
	Ordinal0 [0x004B63BD+1008573]
	Ordinal0 [0x004B6F7C+1011580]
	Ordinal0 [0x004B65CA+1009098]
	Ordinal0 [0x004B5BC6+1006534]
	Ordinal0 [0x004B4AD0+1002192]
	Ordinal0 [0x004B4FAD+1003437]
	Ordinal0 [0x004C5C4A+1072202]
	Ordinal0 [0x0051C19D+1425821]
	Ordinal0 [0x0050B9EC+1358316]
	Ordinal0 [0x0051BAF2+1424114]
	Ordinal0 [0x0050B806+1357830]
	Ordinal0 [0x004E6086+1204358]
	Ordinal0 [0x004E6F96+1208214]
	GetHandleVerifier [0x007DB232+1658114]
	GetHandleVerifier [0x0089312C+2411516]
	GetHandleVerifier [0x006CF261+560433]
	GetHandl

In [ ]:
df=pd.DataFrame(columns=['title','url','pdf'])

df['title']=title
df['url']=url
df['pdf']=pdf

In [ ]:
#내용저장 csv
df.to_csv("theory.csv",header=True, index=False)
df_theory=pd.read_csv('theory.csv')
df_theory

In [ ]:
#os로 파일 다운로드
pdf_url = df['pdf'].values.tolist()
pdf_file = [i.split('/')[-1] for i in df['pdf'].values if 'arxiv' in i]

print(f'전체 파일 수: {len(pdf_url)}')

#enumerate -> index / for n, ( , ) = n=index, ( , )=tuple
#start /b =>os background실행
for n, (url, file) in enumerate(zip(pdf_url, pdf_file)):
    print(n)
    download_path = f'C:/Users/kimEn/Downloads/pdf_download'
    os.system(f'curl "{url}" --output "{download_path}/{file}"')

In [ ]:
#pdf text변환
def pdf_to_txt(pdf_file):
    rsrcmgr = PDFResourceManager()
   
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(pdf_file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,
                                 caching=caching, check_extractable=True):
        interpreter.process_page(page)
    
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    pdf_txt=[]
    pdf_txt.append(text)
    return text

In [ ]:
#오류제외 파일변환

for i in tqdm(pdf_file):    
    try:
        text = pdf_to_txt(f'{download_path}/{i}')
        txt_file_name = '.'.join(i.split('.')[:-1])
        with open(f'{download_path}/txt/{txt_file_name}.txt', 'w', encoding='utf-8') as f:
            f.write(text)
    except Exception as e:
        print(e, i)

In [ ]:
#어간추출
import nltk
from nltk import FreqDist
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer

from glob import glob
from tqdm.notebook import tqdm

def get_wordnet_pos(tagged_pos):
    for pos in ['V', 'N', 'J', 'R']:
        if tagged_pos.startswith(pos):
            return pos.lower() if pos != 'J' else 'a'
    return None


# 어간 추출
def get_stemmer(stemmer):
    "choose stemmer in 'porter' 'lancaster' 'snowball'"
    if stemmer == 'porter':
        stemmer = PorterStemmer()
    elif stemmer == 'lancaster':
        stemmer = LancasterStemmer()
    elif stemmer == 'snowball':
        stemmer = SnowballStemmer('english')
    return stemmer


re_tokenizer = RegexpTokenizer('[a-z]{2,}')
stemmer = get_stemmer('snowball')
lemm = WordNetLemmatizer()
stop_words = stopwords.words('english')

word_token_list = []
for txt_file in tqdm(glob('./Downloads/pdf_download/txt/*.txt')):
    with open(txt_file, encoding='utf-8') as f:
        txt = f.read().lower().replace('-\n', '')
        
    word_tokens = re_tokenizer.tokenize(txt)
    word_tokens = [w for w in word_tokens if w not in stop_words]
    # stemed_txt = [stemmer.stem(w) for w in word_tokens]
    
    word_tokens = pos_tag(word_tokens)
    word_tokens = [(w, get_wordnet_pos(tag)) for w, tag in word_tokens if get_wordnet_pos(tag) != None]
    word_tokens = [lemm.lemmatize(word, pos=tag) for word, tag in word_tokens]
    
    word_token_list.extend(word_tokens)
    
FreqDist(word_token_list)

In [ ]:
with open('./Downloads/pdf_download/txt/1306.5532v2.txt', encoding='utf-8') as f:
    txt = f.read().lower()

# 토크나이저
re_tokenizer = RegexpTokenizer('[a-zA-Z]{2,}')

# 토큰화
word_tokens = re_tokenizer.tokenize(txt)

# 불용어(stopwords) 제거
stop_words = stopwords.words('english')
word_tokens = [w for w in word_tokens if w not in stop_words]

stemmer = PorterStemmer()
# word_tokens = [stemmer.stem(w) for w in word_tokens]

word_tokens = pos_tag(word_tokens)
word_tokens = [(w, get_wordnet_pos(tag)) for w, tag in word_tokens if get_wordnet_pos(tag) != None]
word_tokens = [lemm.lemmatize(word, pos=tag) for word, tag in word_tokens]

FreqDist(word_tokens)

In [ ]:
open_txt=open([f'{download_path}/txt/{i}.txt' for i in txt_file_name],'r+',encoding="UTF-8").read()
word_tokenize(open_txt)

retokenize=RegexpTokenizer('[\w]+')
retoken_txt=retokenize.tokenize(open_txt)

retoken_txt

#어간추출
st1=PorterStemmer()
st2=LancasterStemmer()

print("Porter Stemmer:", [st1.stem(w) for w in retoken_txt])
print("Lancaster Stemmr",[st2.stem(w) for w in retoken_txt])

#원형복원
lm=WordNetLemmatizer()
[lm.lemmatize(w,pos="v") for w in retoken_txt]

#품사 부착
sentence='This is for me the most important part of this thesis, as it would not be possible to do this alone.'
tagged_list=pos_tag(word_tokenize(sentence))
tagged_list

In [ ]:
#태그 튜플 제거
untag(tagged_list)

#같은 토큰, 다른 품사 구분
def tokenizer(doc):
    return ["/".join(p) for p in tagged_list]

tokenizer(sentence)

text=Text(retokenize.tokenize(open_txt))
text.similar("alone") #비슷한문맥의 비슷한단어
text.common_contexts(['alone','context']) #두 단어 공통 문맥

fd=text.vocab()
type(fd)

stopwords=['is','are']
txt_tokens=pos_tag(retokenize.tokenize(open_txt))
names_list=[t[0] for t in txt_tokens if t[1]=="NNP" and t[0] not in stopwords and len(t[0])!=1]
fd_names=FreqDist(names_list)
0
fd_names.most_common(100)

In [ ]:
#빈도수 높은 단어들어간 문장 추출1
import string

f=open('./Downloads/pdf_download/txt/1306.5532v2.txt', encoding='utf-8')
# replace_space(f)
x=f.read()
sentence=x.split('.')
terminology=[]

keywords=FreqDist(word_token_list)

for k in keywords:#3번
    for s in sentence:
        if s.find(k)>0 and ((s.find('is')>0 and t[0] for t in tagged_list if t[1].startswith('V')) or (s.find('are')>0):  #word-token에서 verb만 일치하도록 변경
            terminology.append(s)
            print(s)
        else:
            pass

In [ ]:
#빈도수 높은 단어들어간 문장 추출2 사전저장
import string

keywords=FreqDist(word_tokens)
terminology=[]

for i in txt_file_name:
    f=open(f'{download_path}/txt/{txt_file_name}.txt', 'r', encoding='utf-8')
    x=f.read()
    sentence=x.split('.')

    for k in keywords:#3번
        for s in sentence:
            if s.find(k[0])>0 and s.find('is'):
                terminology.append(s)
                print(s)
            else:
                pass

In [ ]:
#구글번역기로 번역
from googletrans import Translator
keywords=['3d','classification','recognition','predict']
meaning={'3d':'3d explation','classification':'classification explation','recognition':'recognition explation','predict':'prediction explation'}
Ttranslated=[]
for i in meaning.values():
    translator=Translator()
    result=translator.translate(i,src='en', dest='ko')
    Ttranslated.append(result.text)
    print(result)


In [ ]:
#(단어-분류)-(의미 번역-원문-출처) -> csv 저장
import pandas as pd
df2=pd.DataFrame(columns=['단어','설명','원문','출처'])

df2['단어']=meaning.keys()
df2['원문']=meaning.values()
df2['설명']=Ttranslated

df2